In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN", "NAME"], inplace=True)

In [3]:
# Confirm column drop
application_df.columns

Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS',
       'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df["APPLICATION_TYPE"].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types = application_df.groupby("APPLICATION_TYPE").count()
application_types_to_replace = application_types.loc[application_types['IS_SUCCESSFUL'] < 300]
application_types_to_replace = list(application_types_to_replace.index.values)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
application_df["CLASSIFICATION"].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df["CLASSIFICATION"].value_counts()[application_df["CLASSIFICATION"].value_counts() > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_types = application_df.groupby("CLASSIFICATION").count()
classifications_to_replace = classification_types.loc[classification_types['IS_SUCCESSFUL'] < 1800]
classifications_to_replace = list(classifications_to_replace.index.values)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
new_application_df = pd.get_dummies(application_df, dtype=int)

In [11]:
new_application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [12]:
# Split our preprocessed data into our features and target arrays
features = new_application_df.drop(columns="IS_SUCCESSFUL")
target = new_application_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(features,
                                                    target,
                                                    stratify=target)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()


# First  layer
nn.add(tf.keras.layers.Dense(units=86, activation="relu", input_dim=43))

# Hidden Layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 86)                3784      
                                                                 
 dense_1 (Dense)             (None, 30)                2610      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6425 (25.10 KB)
Trainable params: 6425 (25.10 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics="accuracy")

In [16]:
# Create callback that saves model's weights every five epochs as described in TensorFlow documentation https://www.tensorflow.org/tutorials/keras/save_and_load#:~:text=%23%20Create%20a%20callback%20that%20saves%20the%20model%27s%20weights%20every%205%20epochs%0Acp_callback%20%3D%20tf.keras.callbacks.ModelCheckpoint(%0A%C2%A0%20%C2%A0%20filepath%3Dcheckpoint_path%2C%20%0A%C2%A0%20%C2%A0%20verbose%3D1%2C%20%0A%C2%A0%20%C2%A0%20save_weights_only%3DTrue%2C%0A%C2%A0%20%C2%A0%20save_freq%3D5*n_batches)
file_path = "optimization_1/attempt_weights_1"
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath= file_path,
    verbose=1,
    save_weights_only=True,
    save_freq=215)

# Save the weights using the `checkpoint_path` format
nn.save_weights(file_path.format(epoch=0))

In [17]:
# Train the model
nn.fit(X_train_scaled,
       y_train,
       epochs=200,
       callbacks=cp_callback)

Epoch 1/200
211/804 [======>.......................] - ETA: 1s - loss: 0.5893 - accuracy: 0.7050
Epoch 1: saving model to /deep-learning-challenge/download_this_one/first_optimization/attempt_weights_1
416/804 [==============>...............] - ETA: 0s - loss: 0.5836 - accuracy: 0.7076
Epoch 1: saving model to /deep-learning-challenge/download_this_one/first_optimization/attempt_weights_1
625/804 [======================>.......] - ETA: 0s - loss: 0.5730 - accuracy: 0.7160
Epoch 1: saving model to /deep-learning-challenge/download_this_one/first_optimization/attempt_weights_1
804/804 [==============================] - 3s 2ms/step - loss: 0.5683 - accuracy: 0.7204
Epoch 2/200
 52/804 [>.............................] - ETA: 1s - loss: 0.5568 - accuracy: 0.7218
Epoch 2: saving model to /deep-learning-challenge/download_this_one/first_optimization/attempt_weights_1
248/804 [========>.....................] - ETA: 1s - loss: 0.5633 - accuracy: 0.7227
Epoch 2: saving model to /deep-learning-ch

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6442 - accuracy: 0.7332 - 468ms/epoch - 2ms/step
Loss: 0.6442193388938904, Accuracy: 0.7331778407096863


In [19]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


This model included reducing the number of neurons in layer 2 and increasing the number of epochs.
The accuracy for this model went down slightly although the loss also dramatically decreased.
